# In Class Lab 1

### Due by 5 PM Jan 31st in your github repository 'Labs/Lab1' folder

## Part A:  The Local Standard of Rest
Proper motion of Sgr A* from Reid & Brunthaler 2004
$\mu = 6.379$ mas/yr 

Peculiar motion of the sun, $v_\odot$ = 12.24 km/s  (Schonrich 2010)


$v_{tan} = 4.74 \frac{\mu}{\rm mas/yr} \frac{R_o}{\rm kpc} = V_{LSR} + v_\odot$


### a)

Create a function called VLSR to compute the local standard of res (V$_{LSR}$).

The function should take as input: the solar radius (R$_o$), the proper motion (mu)
and the peculiar motion of the sun in the $v_\odot$ direction.

Compute V$_{LSR}$ using three different values R$_o$: 
1. Water Maser Distance for the Sun :  R$_o$ = 8.34 kpc   (Reid 2014 ApJ 783) 
2. GRAVITY Collaboration Distance for the Sun:  R$_o$ = 8.178 kpc   (Abuter+2019 A&A 625)
3. Value for Distance to Sun listed in Sparke & Gallagher : R$_o$ = 7.9 kpc 


In [1]:
# Import Modules 
import numpy as np # import numpy
import astropy.units as u # import astropy units
from astropy import constants as const # import astropy constants
# test
def VLSR(Ro,mu=6.379,vsun=12.24*u.km/u.s):
    '''
    This function will compute the velocity at the local standard of rest
        VLSR = 4.74*mu*R0-vsun
    @param Ro (Quantity), the distance from the sun to the galactic center in kpc
    @param mu (float), the proper motion of Sgr A* in mas/yr, by default from Reid & Brunthaler 2004
    @param vsun (Quanity), the peculiar motion of the sun in the v direction, by default from Schonrich+2010
    @returns vlsr (Quantity), the velocity of the locaity standard of rest
    '''
    return 4.74*mu*(Ro/u.kpc)*u.km/u.s-(vsun)

In [2]:
# Define distances
RoReid = 8.34*u.kpc # From Reid+2014
RoGravity = 8.178*u.kpc # From Abuter+2019
RoSG = 7.9*u.kpc # From Sparke & Gallagher

In [3]:
# Compute using Reid+2014 R0
VLSR_RoReid = VLSR(RoReid)
print(VLSR_RoReid)
# Compute using Abuter+2019
VLSR_RoGravity = VLSR(RoGravity)
print(VLSR_RoGravity)
# Compute using Sparke & Gallagher
VLSR_RoSG = VLSR(RoSG)
print(VLSR_RoSG)

239.9320764 km / s
235.03376988000002 km / s
226.628034 km / s


### b)

compute the orbital period of the sun in Gyr using R$_o$ from the GRAVITY Collaboration (assume circular orbit)

Note that 1 km/s $\sim$ 1kpc/Gyr

In [4]:
def TorbSun(R,V):
    '''
    Computes the orbital period of the sun using T = 2*pi*R/v
    @param R (Quantity), distance to the galacitc center
    @param V (Quantity), velocity of the sun in the v direction
    @returns (Quahttp://localhost:8888/notebooks/Lab1_VLSR.ipynb#ntity), the orbital period of the sun
    '''
    VkpcGyr = V.to(u.kpc/u.Gyr)
    T = 2*np.pi*R/VkpcGyr
    return T

### c)

Compute the number of rotations about the GC over the age of the universe (13.8 Gyr)

In [6]:
vsunPeculiar = 12.24*u.km/u.s
vsun = VLSR_RoGravity + vsunPeculiar
T_grav = TorbSun(RoGravity, vsun) # Compute orbital time of sun

In [7]:
Age = 13.8*u.Gyr
print(Age/T_grav) # Compute number of solar rotations over the age of the universe

67.91779593023313


## Part B  Dark Matter Density Profiles

### a)
Try out Fitting Rotation Curves 
[here](http://wittman.physics.ucdavis.edu/Animations/RotationCurve/GalacticRotation.html)


### b)


In the Isothermal Sphere model, what is the mass enclosed within the solar radius (R$_o$) in units of M$_\odot$? 

Recall that for the Isothermal sphere :
$\rho(r) = \frac{V_{LSR}^2}{4\pi G r^2}$

Where $G$ = 4.4985e-6 kpc$^3$/Gyr$^2$/M$_\odot$, r is in kpc and $V_{LSR}$ is in km/s

What about at 260 kpc (in units of  M$_\odot$) ? 

In [8]:
G = const.G.to(u.kpc**3/u.Gyr**2/u.Msun) # Gets G in desired units

In [10]:
# density profile rho = VLSR^2/(4*pi*G*R^2)
# Mass = Integrate rho dV = rho 4*pi*r**2 dr = VLSR**2 / (4*pi*G*r**2) * (r*pi*G*r**2) dr = VLSR**2/G*R
def MassIso(r,VLSR):
    '''
    Computes dark matter mass enclosed within a given distance from the galactic center, assuming isothermal sphere:
        M = VLSR**2/G*r
    @param r (Quantity), the distance to the galactic center
    @param VLSR (Quantity), the velocity of the local standard of rest
    @returns M (Quantity), the mass envlosed within r
    '''
    VLSRkpcGyr = VLSR.to(u.kpc/u.Gyr) # Convert km/s to kpc/Gyr
    M = VLSRkpcGyr**2/G*r # Mass for isothermal sphere
    return M
MIsoSolar = MassIso(RoGravity, VLSR_RoGravity) # Mass within solar radius
print(f"{MIsoSolar:.2e}")

1.05e+11 solMass


In [12]:
MIso260 = MassIso(260*u.kpc, VLSR_RoGravity) # Mass within 260 kpc
print(f"{MIso260:.2e}")

3.34e+12 solMass


## c) 

The Leo I satellite is one of the fastest moving satellite galaxies we know. 


It is moving with 3D velocity of magnitude: Vtot = 196 km/s at a distance of 260 kpc (Sohn 2013 ApJ 768)

If we assume that Leo I is moving at the escape speed:

$v_{esc}^2 = 2|\Phi| = 2 \int G \frac{\rho(r)}{r}dV $ 

and assuming the Milky Way is well modeled by a Hernquist Sphere with a scale radius of $a$= 30 kpc, what is the minimum mass of the Milky Way (in units of M$_\odot$) ?  

How does this compare to estimates of the mass assuming the Isothermal Sphere model at 260 kpc (from your answer above)

In [18]:
# Hernquist Profile potential: phi = -GM/(r+a)
# v_esc^2/2 = |phi| = GM/(r+a)
# M = v_esc^2(r+a)/(2*G)
def MassFromVesc(vesc,r,a):
    '''
    Determines the total mass needed for a given escape speed assuming a Hernquist Sphere dark matter profile
        M = v_esc^2(r+a)/(2*G)
    @param v_esc (Quantity), the escape speed of an object
    @param r (Quantity), the distance of that object from the center of the galaxy
    @param a (Quantity), the scale radius of the Hernquist Sphere
    @returns M (Quanity), the mass of the galaxy
    '''
    vescKpcGyr = vesc.to(u.kpc/u.Gyr) # Convert velocity units to kpc/Gyr
    return vescKpcGyr**2*(r+a)/(2*G) # Return the required mass

In [19]:
VLeoI = 196*u.km/u.s # Velocity of Leo I from Sohn+2013 ApJ 768
rLeoI = 260*u.kpc # Distance of Leo I from galactic center from Sohn+2013 ApJ 768
a = 30*u.kpc # Scale radius of Hernquist Halo
MLeoI = MassFromVesc(VLeoI, rLeoI, a)
print(f"{MLeoI:2e}")

1.295147e+12 solMass


In [20]:
MIso260/MLeoI # Isothermal sphere mass gets a mass above the calculated limit -- dark matter has to turnover at some point

<Quantity 2.57842045>